<h1>  Project of DDML </h1>

<h3>Author :</h3>
    <ul>
    <li> 
    Thibault POUX
    </li> 
    <li> 
    Pablo García
    </li> 
    <li> 
    Nabil Kaci 
    </li> 
    </ul>

<style>
.container {
    display: flex;
    flex-wrap: wrap;
    justify-content: center;
}

.image-group {
    display: flex;
    flex-direction: row;
    align-items: center;
    margin-bottom: 20px;
}

.image-box img {
    width: 300px; /* Adjust the width as needed */
    height: 300px; /* Adjust the height as needed */
    object-fit: cover;
}

.image-box img {
    max-width: 100%;
    height: auto;
}

.summary {
    margin-top: 20px;
}



.summary li {
    margin-bottom: 10px;
}
</style>

<div class="container">
    <div class="title">
        <h2>MRI Images of brain:</h2>
    </div>
    <div class="image-group">
        <div class="image-box">
            <img src="Dataset\no\no_3.jpg" alt="Image 1">
        </div>
        <div class="image-box">
            <img src="Dataset\no\no_78.jpg" alt="Image 2">
        </div>
    </div>
</div>

<div class="container">
    <div class="title">
        <h2>MRI Images for brain with a tumor:</h2>
    </div>
    <div class="image-group">
        <div class="image-box">
            <img src="Dataset\yes\yes_2.jpg" alt="Image 3">
        </div>
        <div class="image-box">
            <img src="Dataset\yes\yes_12.jpg" alt="Image 4">
        </div>
    </div>
</div>


<div class="summary">
    <h2>Summary</h2>
    <ul>
        <li>Number of cancer images: 155</li>
        <li>Number of non-cancer images: 98</li>
        <li>Link to dataset page: <a href="https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection">Kaggle.com</a></li>
    </ul>
</div>

In [ ]:

#all import 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time

import numpy as np
import matplotlib.pyplot as plt

from tabulate import tabulate


#homemade package
from codigo.prep_data import data_proccesing

#import the data
X_flatten, Y_flatten = data_proccesing.load_data_flatten()




Quadratic Model

In [ ]:
# Variables
r_state = [914, 895, 365, 264, 39, 500, 129]
pourcentage_of_test_data = 0.33

In [ ]:
# specified all the models
reg_Quadra = QuadraticDiscriminantAnalysis()
reg_KNN = KNeighborsClassifier(n_neighbors=3)
reg_MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 3))


f1_scores = {'quadra':[], 'knn':[], 'mlp':[]}
pretty_Names = {'quadra':'Quadratic Discriminant Analysis', 'knn':'K-Nearest Neighbors', 'mlp':'Multi-Layer Perceptron'}
time_counter_fit = {'quadra': 0, 'knn':0, 'mlp':0}
time_counter_test = {'quadra': 0, 'knn':0, 'mlp':0}

for int_state in r_state:

    # make the sample with ouldout
    X_train, X_test, Y_train, Y_test = train_test_split(X_flatten, Y_flatten, test_size=pourcentage_of_test_data, random_state=int_state)

    for reg, name in zip([reg_Quadra, reg_KNN, reg_MLP], f1_scores.keys()): #loop over the models
        start_time = time.time()
        reg = reg.fit(X_train, Y_train)
        end_fit = time.time()
        result = reg.predict(X_test)
        end_predict = time.time()
        
        # save results
        f1_scores[name].append(f1_score(Y_test, result))
        time_counter_fit[name] += end_fit - start_time
        time_counter_test[name] += end_predict - end_fit


# part to display the results

# print the results in a table
output_data = []
for key in f1_scores.keys():
    name = pretty_Names.get(key)
    f1 = np.mean(f1_scores[key])
    time_fit = time_counter_fit[key]/len(r_state)
    time_predict = time_counter_test[key]/len(r_state)
    output_data.append([name, f1, time_fit, time_predict])
    
print(f"Average of the different models, for {len(r_state)} different test, with {pourcentage_of_test_data*100}% of the data used the test set.") 

print(tabulate(output_data, headers=['Model', 'F1 Score', 'Time to fit', 'Time to predict'], tablefmt='grid'))


# plot a graph of the score for each model and for each iterration with a other curve for the average
plt.plot(f1_scores['quadra'], label='Quadratic', color='red')
plt.plot(f1_scores['knn'], label='KNN', color='blue')
plt.plot(f1_scores['mlp'], label='MLP', color='green')
plt.plot([sum(f1_scores['quadra'])/len(f1_scores['quadra'])]*len(f1_scores['quadra']), label='Quadratic average',   color='red', linestyle='--')
plt.plot([sum(f1_scores['knn'])/len(f1_scores['knn'])]*len(f1_scores['knn']), label='KNN average', color='blue', linestyle='--')
plt.plot([sum(f1_scores['mlp'])/len(f1_scores['mlp'])]*len(f1_scores['mlp']), label='MLP average', color='green', linestyle='--')
plt.legend()
plt.show()